In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, os, joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, average_precision_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, LSTM, Flatten, Input, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import optuna
from optuna.integration import TFKerasPruningCallback

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
# ======================================================
# 2️⃣ Carga y preprocesamiento
# ======================================================
path = '../data/processed/ddos_findef.csv'
if not os.path.exists(path):
    raise FileNotFoundError(f"No se encontró el archivo {path}")

df = pd.read_csv(path)
assert 'y' in df.columns, "Debe existir una columna 'y' con 0/1."

y = df['y'].astype(int)
X = df.drop(columns=['y'])

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

In [8]:
# ======================================================
# 3️⃣ Modelos (Baseline, AE+CNN-LSTM, MLP)
# ======================================================

# --- Baseline Logistic Regression ---
lr = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Métricas
def metrics_report(y_true, y_pred, y_proba):
    return {
        'F1_ATTACK': f1_score(y_true, y_pred),
        'Precision_ATTACK': precision_score(y_true, y_pred),
        'Recall_ATTACK': recall_score(y_true, y_pred),
        'Accuracy': accuracy_score(y_true, y_pred),
        'PR_AUC': average_precision_score(y_true, y_proba),
        'ROC_AUC': roc_auc_score(y_true, y_proba)
    }

lr_metrics = metrics_report(y_test, y_pred_lr, lr.predict_proba(X_test_scaled)[:,1])

In [9]:
# ======================================================
# 3️⃣.3 MLP (versión original del notebook)
# ======================================================
mlp = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5)

mlp.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100,
    batch_size=512,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

y_pred_mlp = (mlp.predict(X_test_scaled) > 0.5).astype(int)
y_proba_mlp = mlp.predict(X_test_scaled)
mlp_metrics = metrics_report(y_test, y_pred_mlp, y_proba_mlp)

2025-10-30 20:17:40.199260: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-30 20:17:40.418655: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-30 20:17:40.424005: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


2025-10-30 20:17:41.623265: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-10-30 20:17:41.633506: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 12.0
2025-10-30 20:17:41.633523: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at /home/byd-computer-ffq-u/proyects/Tesis_MIA2025-2/rtxenv/lib/python3.10/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc/bin/ptxas
2025-10-30 20:17:41.633570: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-30 20:17:41.785080: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-10-30 20:17:41.

3638/3638 [==============================] - 13s 2ms/step - loss: 0.1710 - accuracy: 0.9197 - val_loss: 0.1037 - val_accuracy: 0.9624 - lr: 0.0010
Epoch 2/100
3638/3638 [==============================] - 9s 2ms/step - loss: 0.1136 - accuracy: 0.9527 - val_loss: 0.0925 - val_accuracy: 0.9612 - lr: 0.0010
Epoch 3/100
3638/3638 [==============================] - 9s 2ms/step - loss: 0.0983 - accuracy: 0.9602 - val_loss: 0.0882 - val_accuracy: 0.9614 - lr: 0.0010
Epoch 4/100
3638/3638 [==============================] - 8s 2ms/step - loss: 0.0974 - accuracy: 0.9599 - val_loss: 0.0885 - val_accuracy: 0.9638 - lr: 0.0010
Epoch 5/100
3638/3638 [==============================] - 8s 2ms/step - loss: 0.0916 - accuracy: 0.9627 - val_loss: 0.0776 - val_accuracy: 0.9761 - lr: 0.0010
Epoch 6/100
3638/3638 [==============================] - 8s 2ms/step - loss: 0.0915 - accuracy: 0.9626 - val_loss: 0.0666 - val_accuracy: 0.9739 - lr: 0.0010
Epoch 7/100
3638/3638 [==============================] - 8s 2ms

## hiperparámetros con Optuna

In [10]:
def objective(trial):
    n_layers = trial.suggest_int('n_layers', 2, 4)
    n_units = trial.suggest_categorical('n_units', [64, 128, 256])
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    lr = trial.suggest_float('lr', 1e-4, 3e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [256, 512, 1024])

    model = Sequential()
    for _ in range(n_layers):
        model.add(Dense(n_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    pruning = TFKerasPruningCallback(trial, 'val_loss')

    model.fit(
        X_train_scaled, y_train,
        validation_data=(X_val_scaled, y_val),
        epochs=100,
        batch_size=batch_size,
        class_weight=class_weights,
        callbacks=[early_stopping, pruning],
        verbose=0
    )

    y_pred = (model.predict(X_val_scaled) > 0.5).astype(int)
    f1 = f1_score(y_val, y_pred)
    return f1

In [11]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20, timeout=3600)

print('✅ Best trial parameters:')
print(study.best_trial.params)

joblib.dump(study, 'checkpoints/optuna_mlp_study.pkl')

[I 2025-10-30 20:29:51,298] A new study created in memory with name: no-name-c89ff48d-5384-4c77-86cd-b36bb8d3297b


12471/12471 [==============================] - 9s 730us/step


[I 2025-10-30 20:41:26,547] Trial 0 finished with value: 0.9748805598148159 and parameters: {'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.33459233309618475, 'lr': 0.00012645468020907366, 'batch_size': 512}. Best is trial 0 with value: 0.9748805598148159.


12471/12471 [==============================] - 9s 733us/step


[I 2025-10-30 20:43:56,815] Trial 1 finished with value: 0.946972982476704 and parameters: {'n_layers': 2, 'n_units': 256, 'dropout_rate': 0.3441970379588515, 'lr': 0.0009083404663637608, 'batch_size': 512}. Best is trial 0 with value: 0.9748805598148159.


...

[I 2025-10-30 21:05:39,749] Trial 11 finished with value: 0.9655894514543293 and parameters: {'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.41526693331571035, 'lr': 0.00010480992204664538, 'batch_size': 512}. Best is trial 0 with value: 0.9748805598148159.
[I 2025-10-30 21:05:47,716] Trial 12 pruned. Trial was pruned at epoch 0.
[I 2025-10-30 21:05:55,681] Trial 13 pruned. Trial was pruned at epoch 0.
[I 2025-10-30 21:06:05,113] Trial 14 pruned. Trial was pruned at epoch 0.


12471/12471 [==============================] - 9s 744us/step


[I 2025-10-30 21:13:31,624] Trial 15 finished with value: 0.967391432861585 and parameters: {'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.4849023864436812, 'lr': 0.00010443174305099968, 'batch_size': 512}. Best is trial 0 with value: 0.9748805598148159.
[I 2025-10-30 21:13:40,791] Trial 16 pruned. Trial was pruned at epoch 0.
[I 2025-10-30 21:13:48,836] Trial 17 pruned. Trial was pruned at epoch 0.
[I 2025-10-30 21:13:56,857] Trial 18 pruned. Trial was pruned at epoch 0.
[I 2025-10-30 21:14:13,708] Trial 19 pruned. Trial was pruned at epoch 0.


✅ Best trial parameters:
{'n_layers': 2, 'n_units': 128, 'dropout_rate': 0.33459233309618475, 'lr': 0.00012645468020907366, 'batch_size': 512}


['checkpoints/optuna_mlp_study.pkl']

In [12]:
# ======================================================
# 3️⃣.4 Reentrenar el MLP con los mejores hiperparámetros
# ======================================================
params = study.best_trial.params
mlp_final = Sequential()
for _ in range(params['n_layers']):
    mlp_final.add(Dense(params['n_units'], activation='relu'))
    mlp_final.add(Dropout(params['dropout_rate']))
mlp_final.add(Dense(1, activation='sigmoid'))

mlp_final.compile(optimizer=keras.optimizers.Adam(params['lr']), loss='binary_crossentropy', metrics=['accuracy'])
mlp_final.fit(
    np.vstack([X_train_scaled, X_val_scaled]),
    np.concatenate([y_train, y_val]),
    epochs=100,
    batch_size=params['batch_size'],
    class_weight=class_weights,
    callbacks=[EarlyStopping(patience=8, restore_best_weights=True)],
    verbose=1
)

y_pred_optuna = (mlp_final.predict(X_test_scaled) > 0.5).astype(int)
y_proba_optuna = mlp_final.predict(X_test_scaled)
optuna_metrics = metrics_report(y_test, y_pred_optuna, y_proba_optuna)

Epoch 1/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.2947 - accuracy: 0.8229
Epoch 2/100
4417/4417 [==============================] - 8s 2ms/step - loss: 0.2182 - accuracy: 0.8753
Epoch 3/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.1742 - accuracy: 0.9186
Epoch 4/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.1499 - accuracy: 0.9381
Epoch 5/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.1366 - accuracy: 0.9448
Epoch 6/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.1282 - accuracy: 0.9477
Epoch 7/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.1216 - accuracy: 0.9498
Epoch 8/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.1155 - accuracy: 0.9517
Epoch 9/100
4417/4417 [==============================] - 9s 2ms/step - loss: 0.1094 - accuracy: 0.9538
Epoch 10/100
4417/4417 [==============================] - 9s 2ms/step - l

In [13]:
# ======================================================
# 4️⃣ Comparativa final de modelos
# ======================================================
df_results = pd.DataFrame([
    {'Modelo': 'Logistic Regression', **lr_metrics},
    {'Modelo': 'MLP', **mlp_metrics},
    {'Modelo': 'Optuna_MLP', **optuna_metrics}
])
print(df_results)

                Modelo  F1_ATTACK  Precision_ATTACK  Recall_ATTACK  Accuracy  \
0  Logistic Regression   0.652549          0.488406       0.982873  0.781465   
1                  MLP   0.977440          0.960974       0.994479  0.990415   
2           Optuna_MLP   0.980891          0.975184       0.986666  0.991974   

     PR_AUC   ROC_AUC  
0  0.822857  0.935792  
1  0.998320  0.999546  
2  0.998123  0.999484  


In [14]:
# ======================================================
# 5️⃣ Guardado de artefactos
# ======================================================
os.makedirs('checkpoints', exist_ok=True)
joblib.dump(scaler, 'checkpoints/scaler.pkl')
mlp_final.save('checkpoints/mlp_optuna_final.keras')

print("\\n✅ Artefactos guardados en carpeta checkpoints/")


\n✅ Artefactos guardados en carpeta checkpoints/
